In [ ]:
# 原文地址：https://quantpy.com.au/stochastic-volatility-models/heston-model-calibration-to-option-prices/

In [25]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.integrate import quad
from scipy.optimize import minimize 
from datetime import datetime as dt

from eod import EodHistoricalData
from nelson_siegel_svensson import NelsonSiegelSvenssonCurve
from nelson_siegel_svensson.calibrate import calibrate_nss_ols

import fastbox as fb

# 特征方程          
$\varphi\left(X_0, K, v_0, \tau ; \phi\right)=e^{r \phi i \tau} S^{i \phi}\left[\frac{1-g e^{d \tau}}{1-g}\right]^{\frac{-2 a}{\sigma^2}} \exp \left[\frac{a \tau}{\sigma^2}\left(b_2-\rho \sigma \phi i+d\right)+\frac{v_0}{\sigma^2}\left(b_2-\rho \sigma \phi i+d\right)\left[\frac{1-e^{d \tau}}{1-g e^{d \tau}}\right]\right]$        
where     $\mathrm{d}$ and $\mathrm{g}$ no longer change with $\mathrm{b} 1, \mathrm{~b} 2$ or $\mathrm{u} 1, \mathrm{u} 2$      
$$
\begin{aligned}
d & =\sqrt{(\rho \sigma \phi i-b)^2+\sigma^2\left(\phi i+\phi^2\right)} \\
g & =\frac{b-\rho \sigma \phi i+d}{b-\rho \sigma \phi i-d} \\
a & =\kappa \theta \\
b & =\kappa+\lambda
\end{aligned}
$$

In [15]:
def heston_charfunc(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
    
    # constants
    a = kappa*theta
    b = kappa+lambd
    
    # common terms w.r.t phi
    rspi = rho*sigma*phi*1j
    
    # define d parameter given phi and b
    d = np.sqrt( (rho*sigma*phi*1j - b)**2 + (phi*1j+phi**2)*sigma**2 )
    
    # define g parameter given phi, b and d
    g = (b-rspi+d)/(b-rspi-d)
    
    # calculate characteristic function by components
    exp1 = np.exp(r*phi*1j*tau)
    term2 = S0**(phi*1j) * ( (1-g*np.exp(d*tau))/(1-g) )**(-2*a/sigma**2)
    exp2 = np.exp(a*tau*(b-rspi+d)/sigma**2 + v0*(b-rspi+d)*( (1-np.exp(d*tau))/(1-g*np.exp(d*tau)) )/sigma**2)

    return exp1*term2*exp2

# Part 2 – define the integrand as a function         
$\int_0^{\inf } \mathfrak{R}\left[e^{r \tau} \frac{\varphi(\phi-i)}{i \phi K^{i \phi}}-K \frac{\varphi(\phi)}{i \phi K^{i \phi}}\right] d \phi$

In [16]:
def integrand(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
    numerator = np.exp(r*tau)*heston_charfunc(phi-1j,*args) - K*heston_charfunc(phi,*args)
    denominator = 1j*phi*K**(1j*phi)
    return numerator/denominator

# Part 3 – perform numerical integration over integrand and calculate option price
$C\left(S_0, K, v_0, \tau\right)=\frac{1}{2}\left(S_0-K e^{-r \tau}\right)+\frac{1}{\pi} \int_0^{\inf } \mathfrak{R}\left[e^{r \tau} \frac{\varphi(\phi-i)}{i \phi K^{i \phi}}-K \frac{\varphi(\phi)}{i \phi K^{i \phi}}\right] d \phi$

In [17]:
# Using Rectangular Integration
def heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
    
    P, umax, N = 0, 100, 10000
    dphi=umax/N #dphi is width

    for i in range(1,N):
        # rectangular integration
        phi = dphi * (2*i + 1)/2 # midpoint to calculate height
        numerator = np.exp(r*tau)*heston_charfunc(phi-1j,*args) - K * heston_charfunc(phi,*args)
        denominator = 1j*phi*K**(1j*phi)
        
        P += dphi * numerator/denominator
        
    return np.real((S0 - K*np.exp(-r*tau))/2 + P/np.pi)

In [18]:
# Using scipy integrate quad function
def heston_price(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
    
    real_integral, err = np.real( quad(integrand, 0, 100, args=args) )
    
    return (S0 - K*np.exp(-r*tau))/2 + real_integral/np.pi

In [19]:
# Parameters to test model

S0 = 100. # initial asset price
K = 100. # strike
v0 = 0.1 # initial variance
r = 0.03 # risk free rate
kappa = 1.5768 # rate of mean reversion of variance process
theta = 0.0398 # long-term mean variance
sigma = 0.3 # volatility of volatility
lambd = 0.575 # risk premium of variance
rho = -0.5711 # correlation between variance and stock process
tau = 1. # time to maturity

heston_price( S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r )

C:\ProgramData\Anaconda3\lib\site-packages\scipy\integrate\_quadpack_py.py:523: ComplexWarning: Casting complex values to real discards the imaginary part
  return _quadpack._qagse(func,a,b,args,full_output,epsabs,epsrel,limit)


11.540361819355368

In [26]:
yield_maturities = np.array([1/12, 2/12, 3/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])
yeilds = np.array([0.15,0.27,0.50,0.93,1.52,2.13,2.32,2.34,2.37,2.32,2.65,2.52]).astype(float)/100
#NSS model calibrate
curve_fit, status = calibrate_nss_ols(yield_maturities,yeilds) 

curve_fit

NelsonSiegelSvenssonCurve(beta0=0.028391532204406482, beta1=-0.029279498987056646, beta2=0.025428221505665302, beta3=-0.01417407417786765, tau1=0.9922983984869561, tau2=4.781409415241683)

这段代码是一个用于获取股票期权数据并生成波动率曲面的示例代码。下面逐行解释：

4. `market_prices = {}`: 创建一个空字典，用于存储股票期权的市场价格数据。

5. `S0 = resp['lastTradePrice']`: 获取标的资产的最新交易价格，并将其存储在变量 S0 中。

6. `for i in resp['data']:`: 遍历股票期权数据中的每个到期日期。

7. `market_prices[i['expirationDate']] = {}`: 为每个到期日期创建一个子字典，并将其存储在 market_prices 字典中。

8. `market_prices[i['expirationDate']]['strike']`: 将每个到期日期对应的行权价列表存储在子字典中。

9. `market_prices[i['expirationDate']]['price']`: 将每个到期日期对应的期权价格列表存储在子字典中。

10. `all_strikes = [v['strike'] for i,v in market_prices.items()]`: 获取所有到期日期对应的行权价列表，并将它们存储在 all_strikes 列表中。

11. `common_strikes = set.intersection(*map(set,all_strikes))`: 找到所有到期日期共同的行权价，并将其存储在 common_strikes 集合中。

12. `for date, v in market_prices.items()`: 遍历股票期权数据中的每个到期日期和其对应的价格数据。

13. `maturities.append((dt.strptime(date, '%Y-%m-%d') - dt.today()).days/365.25)`: 计算每个到期日期距离当前日期的剩余到期时间（以年为单位），并将其存储在 maturities 列表中。

14. `price = [v['price'][i] for i,x in enumerate(v['strike']) if x in common_strikes]`: 获取每个到期日期共同的行权价对应的期权价格，并将其存储在 price 列表中。

15. `prices.append(price)`: 将每个到期日期共同的行权价对应的期权价格列表存储在 prices 列表中。

16. `price_arr = np.array(prices, dtype=object)`: 将 prices 列表转换为 numpy 数组，并将其存储在 price_arr 中。

17. `volSurface = pd.DataFrame(price_arr, index = maturities, columns = common_strikes)`: 创建一个 pandas DataFrame，用于存储波动率曲面数据。

18. `volSurface = volSurface.iloc[(volSurface.index > 0.04) & (volSurface.index < 1), (volSurface.columns > 3000) & (volSurface.columns < 5000)]`: 对波动率曲面数据进行筛选，只保留剩余到期时间大于 0.04 年且小于 1 年的数据，并且只保留行权价在 3000 到 5000 之间的数据。

19. `volSurface`: 输出经过处理的波动率曲面数据。

In [34]:
option_price = fb.data.indexoptions.daily(instrumentid = 'IO2112',complete = True)

In [38]:
fb.data.indexoptions.info(instrumentid = 'IO2112',complete = True)

,call_put,exchange,name,per_unit,exercise_type,exercise_price,s_month,maturity_date,list_price,list_date,delist_date,last_edate,last_ddate,quote_unit,min_price_chg,zhenzhouid
instrumentid,,,,,,,,,,,,,,,,
IO2112C4200,C,CFFEX,沪深300指数期权2112认购4200,100.0,欧式,4200.0,202112,2021-12-17 15:30:00+08:00,567.8,2021-07-28 09:30:00+08:00,2021-12-17 15:30:00+08:00,2021-12-17 15:30:00+08:00,None,指数点,0.2,IO2112-C-4200
IO2112C4300,C,CFFEX,沪深300指数期权2112认购4300,100.0,欧式,4300.0,202112,2021-12-17 15:30:00+08:00,488.8,2021-07-28 09:30:00+08:00,2021-12-17 15:30:00+08:00,2021-12-17 15:30:00+08:00,None,指数点,0.2,IO2112-C-4300
IO2112C4350,C,CFFEX,沪深300指数期权2112认购4350,100.0,欧式,4350.0,202112,2021-12-17 15:30:00+08:00,507.6,2021-09-22 09:30:00+08:00,2021-12-17 15:30:00+08:00,2021-12-17 15:30:00+08:00,None,指数点,0.2,IO2112-C-4350
IO2112C4400,C,CFFEX,沪深300指数期权2112认购4400,100.0,欧式,4400.0,202112,2021-12-17 15:30:00+08:00,644.2,2020-12-21 09:30:00+08:00,2021-12-17 15:30:00+08:00,2021-12-17 15:30:00+08:00,None,指数点,0.2,IO2112-C-4400
IO2112C4450,C,CFFEX,沪深300指数期权2112认购4450,100.0,欧式,4450.0,202112,2021-12-17 15:30:00+08:00,427.2,2021-09-22 09:30:00+08:00,2021-12-17 15:30:00+08:00,2021-12-17 15:30:00+08:00,None,指数点,0.2,IO2112-C-4450
IO2112C4500,C,CFFEX,沪深300指数期权2112认购4500,100.0,欧式,4500.0,202112,2021-12-17 15:30:00+08:00,584.2,2020-12-21 09:30:00+08:00,2021-12-17 15:30:00+08:00,2021-12-17 15:30:00+08:00,None,指数点,0.2,IO2112-C-4500
IO2112C4550,C,CFFEX,沪深300指数期权2112认购4550,100.0,欧式,4550.0,202112,2021-12-17 15:30:00+08:00,353.6,2021-09-22 09:30:00+08:00,2021-12-17 15:30:00+08:00,2021-12-17 15:30:00+08:00,None,指数点,0.2,IO2112-C-4550
IO2112C4600,C,CFFEX,沪深300指数期权2112认购4600,100.0,欧式,4600.0,202112,2021-12-17 15:30:00+08:00,528.2,2020-12-21 09:30:00+08:00,2021-12-17 15:30:00+08:00,2021-12-17 15:30:00+08:00,None,指数点,0.2,IO2112-C-4600
IO2112C4650,C,CFFEX,沪深300指数期权2112认购4650,100.0,欧式,4650.0,202112,2021-12-17 15:30:00+08:00,287.2,2021-09-22 09:30:00+08:00,2021-12-17 15:30:00+08:00,2021-12-17 15:30:00+08:00,None,指数点,0.2,IO2112-C-4650


In [31]:
# load the key from the environment variables
api_key = os.environ.get('65e6dbd3bad8e3.48891439') #place your api key here as a string,my account have not enough permissions

# create the client instance
client = EodHistoricalData(api_key)

resp = client.get_stock_options('GSPC.INDX')

resp

market_prices = {}

S0 = resp['lastTradePrice']

for i in resp['data']:
    market_prices[i['expirationDate']] = {}
    market_prices[i['expirationDate']]['strike'] = [name['strike'] for name in i['options']['CALL']]# if name['volume'] is not None]
    market_prices[i['expirationDate']]['price'] = [(name['bid']+name['ask'])/2 for name in i['options']['CALL']]# if name['volume'] is not None]

all_strikes = [v['strike'] for i,v in market_prices.items()]
common_strikes = set.intersection(*map(set,all_strikes))
print('Number of common strikes:', len(common_strikes))
common_strikes = sorted(common_strikes)

prices = []
maturities = []

for date, v in market_prices.items():
    maturities.append((dt.strptime(date, '%Y-%m-%d') - dt.today()).days/365.25)
    price = [v['price'][i] for i,x in enumerate(v['strike']) if x in common_strikes]
    prices.append(price)

price_arr = np.array(prices, dtype=object)
np.shape(price_arr)

volSurface = pd.DataFrame(price_arr, index = maturities, columns = common_strikes)
volSurface = volSurface.iloc[(volSurface.index > 0.04) & (volSurface.index < 1), (volSurface.columns > 3000) & (volSurface.columns < 5000)]
volSurface

HTTPError: 401 Client Error: Unauthorized for url: https://eodhistoricaldata.com/api/options/GSPC.INDX?fmt=json

In [ ]:
# Convert our vol surface to dataframe for each option price with parameters
volSurfaceLong = volSurface.melt(ignore_index=False).reset_index()
volSurfaceLong.columns = ['maturity', 'strike', 'price']

# Calculate the risk free rate for each maturity using the fitted yield curve
volSurfaceLong['rate'] = volSurfaceLong['maturity'].apply(curve_fit)

# Parameters to determine via calibration with market prices
$\Theta=(v 0, \kappa, \theta, \sigma, \rho, \lambda)$       
Minimize squared error:
$$
\operatorname{SqErr}(\Theta)=\sum_{i=1}^N \sum_{j=1}^M w_{i j}\left(C_{M P}\left(X_i, \tau_j\right)-C_{S V}\left(S_\tau, X_i, \tau_j, r_j, \Theta\right)\right)^2+\operatorname{Penalty}\left(\Theta, \Theta_0\right)
$$

The penalty function may be e. g. the distance to the initial parameter vector Penalty $\left(\Theta, \Theta_0\right)=\left\|\Theta-\Theta_0\right\|^2$
Calibration - Optimization Objective function
$$
\hat{\Theta}=\underset{\Theta \in U_{\Theta}}{\arg \min } \operatorname{SqErr}(\Theta)
$$

Here we assume that the set of possible combinations of parameters
$U_{\Theta}$ is compact and in the range for which a solution exists.

In [40]:
# This is the calibration function
# heston_price(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)
# Parameters are v0, kappa, theta, sigma, rho, lambd


# Define variables to be used in optimization
S0 = resp['lastTradePrice']
r = volSurfaceLong['rate'].to_numpy('float')
K = volSurfaceLong['strike'].to_numpy('float')
tau = volSurfaceLong['maturity'].to_numpy('float')
P = volSurfaceLong['price'].to_numpy('float')

params = {"v0": {"x0": 0.1, "lbub": [1e-3,0.1]}, 
          "kappa": {"x0": 3, "lbub": [1e-3,5]},
          "theta": {"x0": 0.05, "lbub": [1e-3,0.1]},
          "sigma": {"x0": 0.3, "lbub": [1e-2,1]},
          "rho": {"x0": -0.8, "lbub": [-1,0]},
          "lambd": {"x0": 0.03, "lbub": [-1,1]},
          }

x0 = [param["x0"] for key, param in params.items()]
bnds = [param["lbub"] for key, param in params.items()]

def SqErr(x):
    v0, kappa, theta, sigma, rho, lambd = [param for param in x]
    
    # Attempted to use scipy integrate quad module as constrained to single floats not arrays
    # err = np.sum([ (P_i-heston_price(S0, K_i, v0, kappa, theta, sigma, rho, lambd, tau_i, r_i))**2 /len(P) \
    #               for P_i, K_i, tau_i, r_i in zip(marketPrices, K, tau, r)])
    
    # Decided to use rectangular integration function in the end
    err = np.sum( (P-heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r))**2 /len(P) )
    
    # Zero penalty term - no good guesses for parameters
    pen = 0 #np.sum( [(x_i-x0_i)**2 for x_i, x0_i in zip(x, x0)] )
          
    return err + pen

result = minimize(SqErr, x0, tol = 1e-3, method='SLSQP', options={'maxiter': 1e4 }, bounds=bnds)

v0, kappa, theta, sigma, rho, lambd = [param for param in result.x]
v0, kappa, theta, sigma, rho, lambd

NameError: name 'P' is not defined

# Calculate estimated option prices using calibrated parameters

Using heston model with estimated parameters

In [ ]:
heston_prices = heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)
volSurfaceLong['heston_price'] = heston_prices

# Visualise Market Prices vs Heston Prices

In [ ]:
import plotly.graph_objects as go
from plotly.graph_objs import Surface
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode()

fig = go.Figure(data=[go.Mesh3d(x=volSurfaceLong.maturity, y=volSurfaceLong.strike, z=volSurfaceLong.price, color='mediumblue', opacity=0.55)])

fig.add_scatter3d(x=volSurfaceLong.maturity, y=volSurfaceLong.strike, z=volSurfaceLong.heston_price, mode='markers')

fig.update_layout(
    title_text='Market Prices (Mesh) vs Calibrated Heston Prices (Markers)',
    scene = dict(xaxis_title='TIME (Years)',
                    yaxis_title='STRIKES (Pts)',
                    zaxis_title='INDEX OPTION PRICE (Pts)'),
    height=800,
    width=800
)

fig.show()